In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 4. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
glove_wiki_vectors.most_similar('rain') # SOLUTION

[('rains', 0.8024376034736633),
 ('snow', 0.7520058155059814),
 ('winds', 0.7493997812271118),
 ('downpour', 0.7369566559791565),
 ('fog', 0.7337402105331421),
 ('wind', 0.7243762612342834),
 ('torrential', 0.7123619914054871),
 ('weather', 0.7077540159225464),
 ('storm', 0.6957892775535583),
 ('storms', 0.6879158616065979)]

In [7]:
glove_wiki_vectors.most_similar('meditation') # SOLUTION

[('yoga', 0.7812220454216003),
 ('transcendental', 0.6932344436645508),
 ('mindfulness', 0.692194402217865),
 ('meditative', 0.6846435070037842),
 ('contemplation', 0.6764697432518005),
 ('contemplative', 0.6538777947425842),
 ('spiritual', 0.6215184330940247),
 ('vipassana', 0.6162617206573486),
 ('buddhist', 0.6084271669387817),
 ('taoist', 0.6045625805854797)]

In [8]:
glove_wiki_vectors.most_similar('pomegranate') # SOLUTION

[('apricot', 0.7174664735794067),
 ('mango', 0.7000153660774231),
 ('tamarind', 0.6881566047668457),
 ('pumpkin', 0.6772227883338928),
 ('pear', 0.6767831444740295),
 ('pineapple', 0.6580570340156555),
 ('papaya', 0.6526557803153992),
 ('cranberry', 0.6495745182037354),
 ('peach', 0.6447383165359497),
 ('watermelon', 0.6376971006393433)]

In [9]:
glove_wiki_vectors.most_similar('ipad') # SOLUTION

[('iphone', 0.9220295548439026),
 ('ipod', 0.831965446472168),
 ('smartphone', 0.7812002301216125),
 ('android', 0.7511346936225891),
 ('app', 0.743316650390625),
 ('ios', 0.7431976795196533),
 ('smartphones', 0.7359461784362793),
 ('apps', 0.7259055972099304),
 ('iphones', 0.7074244022369385),
 ('blackberry', 0.6987718343734741)]

In [10]:
glove_wiki_vectors.most_similar('cobain') # SOLUTION

[('nirvana', 0.7144757509231567),
 ('zeppelin', 0.5535445213317871),
 ('grunge', 0.5420762896537781),
 ('grohl', 0.5335221886634827),
 ('kurt', 0.5284802913665771),
 ('hendrix', 0.5253633260726929),
 ('jimi', 0.5184423923492432),
 ('nevermind', 0.5024071335792542),
 ('frontman', 0.4990401864051819),
 ('mellencamp', 0.49823328852653503)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
# BEGIN SOLUTION
for (w1, w2) in word_pairs:
    print(
        "Cosine similarity between %s and %s = %0.3f"
        % (w1, w2, glove_wiki_vectors.similarity(w1, w2))
    )
# END SOLUTION    

Cosine similarity between coast and shore = 0.700
Cosine similarity between clothes and closet = 0.546
Cosine similarity between old and new = 0.643
Cosine similarity between smart and intelligent = 0.755
Cosine similarity between dog and cat = 0.880
Cosine similarity between tree and lawyer = 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
# BEGIN SOLUTION
for word in test_words:
    print("word {} is in the vocab: {}".format((word), (word in glove_wiki_vectors)))
# END SOLUTION    

word covididiot is in the vocab: False
word fomo is in the vocab: False
word frenemies is in the vocab: True
word anthropause is in the vocab: False
word photobomb is in the vocab: False
word selfie is in the vocab: False
word pxg is in the vocab: False
word pacg is in the vocab: False
word cct is in the vocab: True
word escc is in the vocab: True


In [15]:
grader.check("q1.3")

q1.3 results: All test cases passed!

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [16]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [17]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [18]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [19]:
glove_wiki_vectors.similarity("white", "success")

0.35182396

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [20]:
analogy("man", "intelligent", "woman")  # SOLUTION

man : intelligent :: woman : ?


,Analogy word,Score
0,articulate,0.627857
1,thoughtful,0.616820
2,smart,0.587221
3,strong-willed,0.568862
4,vivacious,0.550627
5,opinionated,0.547024
6,perceptive,0.545705
7,inquisitive,0.544539
8,energetic,0.533960
9,literate,0.533894


In [21]:
analogy("man", "manager", "woman") # SOLUTION

man : manager :: woman : ?


,Analogy word,Score
0,assistant,0.656641
1,job,0.626392
2,supervisor,0.593479
3,owner,0.587832
4,hired,0.577390
5,consultant,0.570659
6,director,0.565653
7,office,0.564568
8,employee,0.561287
9,ceo,0.556422


In [22]:
analogy("man", "engineer", "woman") # SOLUTION

man : engineer :: woman : ?


,Analogy word,Score
0,technician,0.662020
1,educator,0.611515
2,engineers,0.592235
3,surgeon,0.588020
4,contractor,0.587330
5,engineering,0.568600
6,nurse,0.556342
7,chemist,0.554719
8,pioneer,0.546008
9,physician,0.545960


In [23]:
analogy("white", "innocent", "black") # SOLUTION

white : innocent :: black : ?


,Analogy word,Score
0,murder,0.611457
1,civilians,0.602935
2,guilty,0.598535
3,defenseless,0.590402
4,victims,0.586925
5,victim,0.577593
6,innocents,0.575223
7,motivated,0.574008
8,murdering,0.569938
9,humanity,0.569532


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

1. Although these embeddings are de-biased, we still see some stereotypes such as "men" are more associated with managerial roles whereas women are associated with assistant roles. Also, white is associated with innocent whereas black is associated with murder. We also see that white is closer to "success" than "aboriginal". That said, the similarity scores also depend upon the frequency of words, and if a certain word does not occur very frequently in the corpus, the similarity scores are going to be low for that word. This might be the case for the word aboriginal.    

2. Perhaps NLP-based resume screening with such a model could lead to gender bias, say if it was used for hiring a computer programmer, which the model associates with men.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [24]:
from sklearn.datasets import fetch_20newsgroups

In [25]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [26]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [27]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [28]:
# BEGIN SOLUTION

import re
def preprocess_spacy(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    text : (str)
        text to be preprocessed
    min_token_len : (int)
        minimum token length required to keep a token in the preprocessed text
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """
    clean_text = []

    for token in doc:
        if token.like_email:  # Check if the token seems to be like a url
            clean_text.append("EMAIL")
        elif token.like_url:  # Check if the token seems to be like a url
            clean_text.append("URL")
        elif token.like_num:  # Check if the token seems to be like a number
            clean_text.append("NUM")
        elif (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            clean_text.append(token.lemma_.lower())
    return " ".join(clean_text)

# END SOLUTION

In [29]:
# BEGIN SOLUTION
def preprocess(text):
    # Replace a sequence of whitespaces by a single whitespace
    text = re.sub(r"\s+", " ", text)

    # Remove other strange characters
    text = re.sub(r"""[\n\r]+""", "", text)

    # Remove other strange characters
    text = re.sub(r"""[\*\~]+""", "", text)

    # Replace slashes with spaces
    text = re.sub(r"""[\/]+""", " ", text)

    return text

df["text"] = df["text"].apply(preprocess)
df.dropna()

# END SOLUTION

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,It's not a bad question: I don't have any ref...,1,comp.graphics
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics
3,"The following problem is really bugging me, an...",2,comp.windows.x
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone :: I am looking for some software ...,1,comp.graphics
4559,Archive-name: x-faq part3 Last-modified: 1993 ...,2,comp.windows.x
4560,"That's nice, but it doesn't answer the questi...",6,talk.politics.guns
4561,"Hi, I just got myself a Gateway 4DX-33V and tr...",2,comp.windows.x


In [30]:
# BEGIN SOLUTION
df["text_pp"] = [
    preprocess_spacy(spacy_text) for spacy_text in nlp.pipe(df["text"].tolist())
]
# END SOLUTION

In [31]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"Actuallay I don't, but on the other hand I do...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me, an...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,This is the latest from UPI Foreign Ministry ...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi, I'd like to subscribe to Leadership Magazi...",5,soc.religion.christian,like subscribe leadership magazine wonder NUM ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

_Type your answer here, replacing this text._

I decided to carry out the following preprocessing steps

- getting rid of slashes and new-line characters etc.
- sentence segmentation and tokenization      
- replaced all urls with token "URL" emails with token "EMAIL" and numbers with token "NUM" because these tokens in their original form are going to be quite unique. 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluded stopwords and punctuation because they are ubiquitous and unlikely to help us in topic modeling.  
- lemmatization   

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [32]:
# BEGIN SOLUTION NO PROMPT
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(df["text_pp"])
X_vec = vec.transform(df["text_pp"])
# END SOLUTION

In [33]:
# BEGIN SOLUTION NO PROMPT
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=123
)
document_topics = lda.fit_transform(X_vec)
# END SOLUTION

I picked 4 topics because in the categories we have chosen from this dataset, it seems like there could be four broad categories: religion, computers, sports, politics. Also, in my trial and error, I got the most sensible results with 4 topics on this dataset.  

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [34]:
# BEGIN SOLUTION NO PROMPT

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

# END SOLUTION

In [35]:
# BEGIN SOLUTION NO PROMPT
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=20,
)
# END SOLUTION

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           file          num           num           
god           num           people        url           
know          program       say           email         
people        entry         know          game          
think         image         think         team          
believe       output        go            run           
jesus         use           gun           play          
say           jpeg          year          window        
question      color         time          player        
time          section       come          available     
thing         format        armenian      server        
way           url           right         win           
good          write         kill          year          
point         line          want          new           
find          read          state         good          
come          bit           tel

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [36]:
# BEGIN SOLUTION NO PROMPT
for i in range(5):
    print('\n\nDOCUMENT', df.iloc[i]['text'])
    print("\n\nDOCUMENT TOPIC ASSIGNMENT: ", document_topics[i]) 
    print('-----------')
# END SOLUTION    



DOCUMENT You know, I was reading 18 U.S.C. 922 and something just did not make sence and I was wondering if someone could help me out. Say U.S.C. 922 : (1) Except as provided in paragraph (2), it shall be unlawful for any person to transfer or possess a machinegun. Well I got to looking in my law dictionary and I found that a "person" might also be an artificial entity that is created by government and has no rights under the federal constitution. So, what I don't understand is how a statute like 922 can be enforced on an individual. So someone tell me how my government can tell me what I can or cannot possess. Just passing a law does not make it LAW. Everyone knows that laws are constitional until it goes to court. So, has it ever gone to court, not just your run of the mill "Ok I had it I am guilty, put me in jail" Has anyone ever claimed that they had a right to possess and was told by the Supreme Court that they didn't have that right? 


DOCUMENT TOPIC ASSIGNMENT:  [0.0043715  0

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing or computer vision? Briefly explain.     
4. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
5. In Lecture 18, we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

1. Content-based filtering operates under the assumption that item features are available. It uses these features along with ratings data to construct individual user profiles and employs supervised machine learning models to predict ratings.
2. TBD
3. In the context of NLP and vision, training models from scratch is time consuming and resource intensive. A common practice is to download powerful pre-trained models which capture rich information about language and images and tuning these models for our specific tasks. This is called transfer learning.
4. Here is how each model might be tackling multiclass classification. 
    - Decision trees: Natural extension to multi-class. The formula to calculate Gini index or information gain work for multiclass. 
    - KNNs: Natural extension to multiclass. We can just take a vote among multiple classes. 
    - RBF SVMs: We haven't talked about it yet but RBF SVMs use a kernel trick, which corresponds to transforming the original features into a new feature space and fitting a boundary there. But it's still dividing the boundary in half so we need multiclass meta strategy here. The default strategy used in `sklearn` is one-vs.-one. See [here](https://scikit-learn.org/stable/modules/svm.html#multi-class-classification).   
    - Logistic regression: Above, it's using one-vs.-rest approach to handle multi-class. In this strategy a binary model is learned for each class that tries to separate that class from all of the other classes, resulting in as many binary models as there are classes. Note that there is also a multinomial logistic regression also called as **the maxent classifier**, which is different than the ovr meta strategy we are using above.
5. Because we could keep most of the transformations, and just replace the last one with something tailored to our task. 

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)